# Tutorial 5: The Least Fixed Point

*The Kleene Fixed Point Theorem*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-continuous-domains/blob/main/notebooks/05-the-least-fixed-point.ipynb)

---

## A Note from the Archives

> *Year 867, Capital Archives*
>
> Arren Mott's pen moved swiftly across the page. After twelve years of work, he had found the theorem that would solve the recursion problem.
>
> "Let f be any continuous function on a dcpo D with bottom," he wrote. "Then f has a least fixed point, and it can be computed as:"
>
> $$fix(f) = \bigsqcup_{n \geq 0} f^n(\bot)$$
>
> "The construction is simple: start with ⊥, apply f, apply f again, take the limit. The limit is the least fixed point."
>
> He paused, then added: "This gives meaning to every recursive definition. The Y combinator, which types forbid, now has a semantic counterpart: the fixed-point operator."
>
> This was EV-867-001: the discovery of the least fixed point theorem.

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. State the least fixed point theorem
2. Compute fixed points by iteration from ⊥
3. Explain why the construction yields a fixed point
4. Understand why it is the LEAST fixed point
5. Apply fixed-point semantics to recursive definitions

## Setup

In [ ]:
import pandas as pd
import numpy as np

# Load datasets
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

fixpoints_df = pd.read_csv(BASE_URL + "fixed_point_computations.csv")
functions_df = pd.read_csv(BASE_URL + "continuous_functions.csv")

print(f"Loaded {len(fixpoints_df)} fixed point computations")

## 1. The Least Fixed Point Theorem

**Theorem** (Kleene/Mott): Let D be a pointed dcpo and f: D → D be continuous. Then f has a least fixed point:

$$fix(f) = \bigsqcup_{n \geq 0} f^n(\bot) = \bot \sqcup f(\bot) \sqcup f(f(\bot)) \sqcup \ldots$$

This is the key theorem of domain theory.

In [ ]:
# The fixed point construction
print("The Fixed Point Construction:")
print()
print("  f⁰(⊥) = ⊥")
print("  f¹(⊥) = f(⊥)")
print("  f²(⊥) = f(f(⊥))")
print("  f³(⊥) = f(f(f(⊥)))")
print("  ...")
print()
print("  fix(f) = ⊔{fⁿ(⊥) | n ≥ 0}")

## 2. Example: Factorial

Consider the recursive definition:
```
factorial(n) = if n = 0 then 1 else n * factorial(n-1)
```

This is the fixed point of the functional:
```
F = λf.λn. if n = 0 then 1 else n * f(n-1)
```

Let's compute fix(F) by iteration!

In [ ]:
# Factorial fixed point computation
factorial_fp = fixpoints_df[fixpoints_df['function_name'] == 'factorial'].sort_values('iteration')

print("Computing factorial as a fixed point:")
print()
print("| Iteration | Input | Value | Notes |")
print("|-----------|-------|-------|-------|")

for _, row in factorial_fp.iterrows():
    input_val = row['input_value'] if row['input_value'] != "any" else "*"
    print(f"| {row['iteration']:>9} | {input_val:>5} | {row['current_approximation']:>5} | {row['notes'][:40]} |")

## 3. Why This Works

**Step 1**: The sequence forms a chain.

Since f is monotone and ⊥ ⊑ f(⊥):
```
⊥ ⊑ f(⊥) ⊑ f(f(⊥)) ⊑ f(f(f(⊥))) ⊑ ...
```

**Step 2**: The chain has a supremum.

Because D is a dcpo, ⊔{fⁿ(⊥)} exists.

**Step 3**: The supremum is a fixed point.

Because f is continuous:
```
f(⊔{fⁿ(⊥)}) = ⊔{f(fⁿ(⊥))} = ⊔{fⁿ⁺¹(⊥)} = ⊔{fⁿ(⊥)}
```

In [ ]:
# Demonstrate the chain property
print("The iteration forms an increasing chain:")
print()
print("  f⁰(⊥) = ⊥ = {everywhere undefined}")
print("  f¹(⊥) = {0→1, else→⊥}")
print("  f²(⊥) = {0→1, 1→1, else→⊥}")
print("  f³(⊥) = {0→1, 1→1, 2→2, else→⊥}")
print()
print("Each approximation ⊑ the next: more inputs are defined.")
print("The limit defines factorial for ALL inputs.")

## 4. Why It's the LEAST Fixed Point

Any fixed point x satisfies f(x) = x.

Since ⊥ ⊑ x and f is monotone:
```
⊥ ⊑ x
f(⊥) ⊑ f(x) = x
f(f(⊥)) ⊑ f(x) = x
...
```

So every fⁿ(⊥) ⊑ x, which means ⊔{fⁿ(⊥)} ⊑ x.

**The iterated fixed point is below any other fixed point.**

In [ ]:
# The least fixed point is minimal
print("Why 'least' fixed point?")
print()
print("  If g is ANY fixed point of F (so F(g) = g), then fix(F) ⊑ g.")
print()
print("  The iterative construction starts from ⊥ — the minimum.")
print("  It builds up just enough to satisfy F(x) = x.")
print("  It doesn't add any 'extra' information.")

## 5. Example: Fibonacci

```
fib(n) = if n ≤ 1 then n else fib(n-1) + fib(n-2)
```

This is fix(F) where:
```
F = λf.λn. if n ≤ 1 then n else f(n-1) + f(n-2)
```

In [ ]:
# Fibonacci fixed point
fib_fp = fixpoints_df[fixpoints_df['function_name'] == 'fibonacci'].sort_values('iteration')

print("Computing fibonacci as a fixed point:")
print()
for _, row in fib_fp.iterrows():
    print(f"  Iteration {row['iteration']}: fib({row['input_value']}) = {row['current_approximation']}")

## 6. Example: Omega

What about Omega? It's the fixed point of:
```
F = λx.x  (the identity)
```

The iteration:
```
f⁰(⊥) = ⊥
f¹(⊥) = ⊥
f²(⊥) = ⊥
...
```

The fixed point is ⊥! Omega's meaning IS bottom — no information.

In [ ]:
# Omega's fixed point is bottom
omega_fp = fixpoints_df[fixpoints_df['function_name'] == 'omega']

print("Omega as a fixed point:")
print()
if not omega_fp.empty:
    for _, row in omega_fp.iterrows():
        print(f"  Iteration {row['iteration']}: {row['current_approximation']}")
        print(f"  Is fixed point: {row['is_fixed_point']}")
        print(f"  Note: {row['notes']}")

## 7. The Fix Operator

We can define a **fix operator** on any dcpo:

```
fix: ((D → D) → D)
fix(f) = ⊔{fⁿ(⊥) | n ≥ 0}
```

This is the domain-theoretic analog of the Y combinator!

- Y combinator: syntactic fixed point (untyped)
- fix operator: semantic fixed point (can be typed)

In [ ]:
# Compare Y and fix
print("Y combinator vs fix operator:")
print()
print("  Y = λf.((λx.f(x x))(λx.f(x x)))")
print("    - Syntactic: manipulates lambda terms")
print("    - Untyped: uses self-application")
print("    - Y f →* f(Y f) by reduction")
print()
print("  fix = λf.⊔{fⁿ(⊥)}")
print("    - Semantic: operates on functions in domains")
print("    - Typable: fix: ((τ → τ) → τ)")
print("    - fix(f) = f(fix(f)) by the fixed point theorem")

## 8. Application: Recursive Types

The fixed point theorem also gives meaning to **recursive types**!

```
List = 1 + A × List
```

This is a type equation. The solution is:
```
List = fix(λT. 1 + A × T)
```

Domain theory provides foundations for recursive data types.

In [ ]:
# Infinite list as fixed point
ones_fp = fixpoints_df[fixpoints_df['function_name'] == 'ones_list'].sort_values('iteration')

print("Infinite list of ones as a fixed point:")
print()
print("  ones = 1 : ones")
print("  ones = fix(λxs. 1 : xs)")
print()
for _, row in ones_fp.iterrows():
    print(f"  Iteration {row['iteration']}: {row['current_approximation']}")

## Summary

In this tutorial, we learned:

1. **The Least Fixed Point Theorem**: fix(f) = ⊔{fⁿ(⊥)}
2. **Iteration from ⊥** builds up the fixed point
3. **Continuity** ensures the limit is a fixed point
4. **Least** because it's built from ⊥ with no extra information
5. **Factorial and Fibonacci** as fixed points
6. **Omega's fixed point is ⊥** — non-termination has meaning
7. **The fix operator** is a typed version of Y

In the next tutorial, we'll see how to use domains to give semantics to lambda calculus.

---

## Exercises

1. Compute the first 4 iterations for fix(λf.λn. if n=0 then 0 else n + f(n-1)).

2. What is fix(λx.⊥)? What is fix(λx.42)?

3. Why does the fixed point theorem require D to have a bottom element?

4. Can a continuous function have multiple fixed points? Give an example.

---

*Next: Tutorial 6 - Denotational Semantics*